In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
working_dir = '/Users/adam/phd/projects/certificates/data_dir/'

df = pd.read_csv('/Users/adam/phd/projects/certificates/dataframe.csv')
df.scheme = df.scheme.astype('category').cat.codes
df.cert_lab = df.cert_lab.astype('category').cat.codes
df.category = df.category.astype('category').cat.codes
df.drop(columns=['Unnamed: 0', 'sec_level', 'cert_date', 'n_updates', 'n_pages', 'manufacturer', 'archived_date'], inplace=True)

In [ ]:
data = df.loc[:, ['category', 'scheme', 'cert_lab', 'n_pages_cat', 'n_updates_cat', 'cert_year']]
tf_data = tf.Variable(data.values)
labels = df.sec_level_cat.tolist()
np.savetxt(os.path.join(working_dir, 'metadata.tsv'), labels, delimiter='\t')

In [ ]:
with tf.Session() as sess:
    saver = tf.train.Saver([tf_data])
    sess.run(tf_data.initializer)
    saver.save(sess, os.path.join(working_dir, 'tf_data.cpkt'))
    config = projector.ProjectorConfig()
    
    embedding = config.embeddings.add()
    embedding.tensor_name = tf_data.name

    # Link this tensor to its metadata(Labels) file
    embedding.metadata_path = 'metadata.tsv'
        
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(working_dir), config)


In [ ]:
data.columns